In [8]:

import requests
from pprint import pprint
import pandas as pd
import json
 
def write_time_column(data):
    data["title"].append("Year")
    data["name"].append("year")
    data["component_attachment"].append("qb:dimension")
    data["property_template"].append("http://purl.org/linked-data/sdmx/2009/dimension#refPeriod")
    data["value_template"].append("http://reference.data.gov.uk/id/year/{year}")
    data["datatype"].append("string")
    data["value_transformation"].append("")
    data["regex"].append("")
    data["range"].append("")
    
    return data
        
        
def write_admin_geography(data):
    data["title"].append("Geography")
    data["name"].append("geography")
    data["component_attachment"].append("qb:dimension")
    data["property_template"].append("http://purl.org/linked-data/sdmx/2009/dimension#refArea")
    data["value_template"].append("http://statistics.data.gov.uk/id/statistical-geography/{geography}")
    data["datatype"].append("string")
    data["value_transformation"].append("")
    data["regex"].append("[A-Z][0-9]{8}")
    data["range"].append("")
    
    return data

    
# TODO - these should be automatically included in the new columns.csv's
# code lists that we dont want to convert for...reasons
NON_STANDARD = {
    "calendar-years": {
        "write_column": write_time_column
    },
    "admin-geography": {
        "write_column": write_admin_geography
    }
}

dataset_url_list = [
    "https://api.beta.ons.gov.uk/v1/datasets/ashe-tables-7-and-8"
]

def get_unique_codelist_urls_from_a_dataset(url):
    
    codelist_list = []
    
    # Get the dataset info
    r = requests.get(url)
    if r.status_code != 200:
        raise ValueError("Failed with status code: " + r.status_code)
        
    dataset_as_dict = r.json()
    lastest_version_url = dataset_as_dict["links"]["latest_version"]["href"]
    
    #  Get the lastest version of that dataset info
    r = requests.get(lastest_version_url)
    if r.status_code != 200:
        raise ValueError("Failed with status code: " + r.status_code)
                         
    lastest_version_as_dict = r.json()
    
    # For each dimension
    for dimension in lastest_version_as_dict["dimensions"]:
        
        code_list_url = "https://api.beta.ons.gov.uk/v1/code-lists/{}/editions/one-off/codes".format(dimension["id"])
        codelist_list.append(code_list_url)
    
    return codelist_list


def create_codelist_reference_csv_from_codelist_url(url):
    
    #  Get the lastest version of that dataset info
    r = requests.get(url)
    if r.status_code != 200:
        raise ValueError("Failed with status code: " + r.status_code)
                        
    code_list_info = r.json()
    
    # Get the codelist id
    code_list_id = code_list_info["items"][0]["links"]["code_list"]["href"].split("/")[-1]
    
    if code_list_id in NON_STANDARD.keys():
        return 
    
    df_dict = {
        "Label":[],
        "Notation":[],
        "Parent Notation":[],
        "Sort Priority":[]
    }
    
    for code_list in code_list_info["items"]:
        
        df_dict["Label"].append(code_list["label"])
        df_dict["Notation"].append(code_list["id"])
        df_dict["Parent Notation"].append("")
        df_dict["Sort Priority"].append("")
                    
    df = pd.DataFrame().from_dict(df_dict)
    df.to_csv("../reference/codelists/{}.csv".format(code_list_id), index=False)
                   

def get_list_of_code_list_url_for_list_of_datasets(dataset_url_list):
    
    all_code_lists = []
    for dataset in dataset_url_list:
        code_list_urls_from_dataset = get_unique_codelist_urls_from_a_dataset(dataset)
    
    for code_list_url in code_list_urls_from_dataset:
        if code_list_url not in all_code_lists:
            all_code_lists.append(code_list_url)
            
    return all_code_lists


def populate_codelist_reference_csvs_from_codelist_urls(code_list_urls_from_dataset):
    
    for cl in code_list_urls_from_dataset:
        create_codelist_reference_csv_from_codelist_url(cl)
        

def components_csv_from_list_of_code_list_urls(code_list_urls_from_dataset):
    
    columns=["Label", "Description", "Component Type", "Codelist"]
    try:
        df = pd.read_csv("../reference/components.csv")
    except FileNotFoundError:
        df = pd.DataFrame(columns=columns)
    except:
        raise
      
    # adding in attributes for measure type
    data = {
        "Label":["Paid hours worked", "Pay GBP"],
        "Description":["", ""],
        "Component Type":["Measure", "Measure"],
        "Codelist":["", ""]
    }
    
    for code_list_url in code_list_urls_from_dataset:
        
        # Get the codelist id
        code_list_id = code_list_url.split("/")[-4]
    
        # Where non standard handling is required, call it then go to next code list
        if code_list_id in NON_STANDARD.keys():
            continue
            
        data["Label"].append(code_list_id.replace("-", " "))
        data["Description"].append("")
        data["Component Type"].append("Dimension")
        data["Codelist"].append("http://gss-data.org.uk/def/concept-scheme/"+code_list_id)
        
    df = pd.concat([df, pd.DataFrame().from_dict(data)])
    df = df.fillna("")
    
    df = df.drop_duplicates()
    
    df.to_csv("../reference/components.csv", index=False)


def populate_columns_csv_from_list_of_code_list_urls(code_list_urls_from_dataset):
    
    columns=["title", "name", "component_attachment", "property_template", "value_template",
                                   "datatype", "value_transformation", "regex", "range"]
    try:
        df = pd.read_csv("../reference/columns.csv")
    except FileNotFoundError:
        df = pd.DataFrame(columns=columns)
    except:
        raise
        
    # Start with Value as default (have to have a value definition)
    # Value,value,,http://gss-data.org.uk/def/measure/{measure_type},,number,,,
    # Marker,marker,qb:attribute,http://purl.org/linked-data/sdmx/2009/attribute#obsStatus,http://gss-data.org.uk/def/concept/marker/{marker},string,,,http://gss-data.org.uk/def/classes/gp-practice/marker
    # Upper CI,upper_ci,qb:attribute,http://gss-data.org.uk/def/attribute/upper-ci,,number,,,
    # Measure Type,measure_type,qb:dimension,http://purl.org/linked-data/cube#measureType,http://gss-data.org.uk/def/measure/{measure_type},string,slugize,,qb:MeasureProperty
    data = {
        "title":["Value", "Marker", "CV", "Measure Type", "Paid hours worked", "Pay GBP"],
        "name":["value", "marker", "cv", "measure_type", "paid_hours_worked", "pay_gbp"],
        "component_attachment":["", "qb:attribute", "qb:attribute", "qb:dimension", "qb:measure", "qb:measure"],
        "property_template":["http://gss-data.org.uk/def/measure/{measure_type}", "http://purl.org/linked-data/sdmx/2009/attribute#obsStatus", "http://gss-data.org.uk/def/attribute/cv", "http://purl.org/linked-data/cube#measureType","http://gss-data.org.uk/def/measure/paid-hours-worked", "http://gss-data.org.uk/def/measure/pay-gbp"],
        "value_template":["", "http://gss-data.org.uk/def/concept/marker/{marker}", "", "http://gss-data.org.uk/def/measure/{measure_type}", "", ""],
        "datatype":["number", "string", "string", "string", "string", "string"],
        "value_transformation":["", "", "", "slugize", "", ""],
        "regex":["", "", "", "", "", ""],
        "range":["", "", "", "", "", ""]
    }
    
    for code_list_url in code_list_urls_from_dataset:
        
        # Get the codelist id
        code_list_id = code_list_url.split("/")[-4]
    
        # Where non standard handling is required, call it then go to next code list
        if code_list_id in NON_STANDARD.keys():
            data = NON_STANDARD[code_list_id]["write_column"](data)
        else:

            data["title"].append(code_list_id.replace("-", " "))
            data["name"].append(code_list_id.replace("-", "_"))
            data["component_attachment"].append("qb:dimension")
            data["property_template"].append("http://gss-data.org.uk/def/dimension/"+code_list_id)
            data["value_template"].append("http://gss-data.org.uk/def/dimension/"+code_list_id+"/{"+code_list_id.replace("-", "_")+"}")
            data["datatype"].append("string")
            data["value_transformation"].append("slugize")
            data["regex"].append("")
            data["range"].append("http://gss-data.org.uk/def/classes/"+code_list_id+"/"+code_list_id)
        
    df = pd.concat([df, pd.DataFrame().from_dict(data)])
    df = df.fillna("")
    
    df = df.drop_duplicates()
    
    df.to_csv("../reference/columns.csv", index=False)
    
def codelists_metadata_from_list_of_code_lists(code_list_urls_from_dataset):
    
    tables = []
    
    for code_list_url in code_list_urls_from_dataset:
        
        # Get the codelist id
        code_list_id = code_list_url.split("/")[-4]
    
        # Where non standard handling is required, call it then go to next code list
        if code_list_id in NON_STANDARD.keys():
            continue
            
        table = {
            "url": "codelists/{}.csv".format(code_list_id),
            "tableSchema": "https://gss-cogs.github.io/ref_common/codelist-schema.json",
            "rdfs:label": code_list_id.replace("-", " ")
        }
        
        tables.append(table)
    
    metadata_dict = {
      "@context": ["http://www.w3.org/ns/csvw", {"@language": "en"}],
      "tables": tables
    }
        
    with open("../reference/codelists-metadata.json" , "w") as f:
        json.dump(metadata_dict, f)
        
        
dataset_url_list = [
    "https://api.beta.ons.gov.uk/v1/datasets/ashe-tables-7-and-8"
]

# Populate /reference/codelists
code_list_urls_from_dataset = get_list_of_code_list_url_for_list_of_datasets(dataset_url_list)
populate_codelist_reference_csvs_from_codelist_urls(code_list_urls_from_dataset)

# Populuate ../reference/columns.csv
populate_columns_csv_from_list_of_code_list_urls(code_list_urls_from_dataset)
        
# Populate ../reference/components.csv
components_csv_from_list_of_code_list_urls(code_list_urls_from_dataset)

# Populate ../reference/codelists-metadata.json
codelists_metadata_from_list_of_code_lists(code_list_urls_from_dataset)

